In [1]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.nc.*
GlobalConstants.setVerbose(VerboseLevel.STD);

In [2]:
// Block 1: Create network and node
val model = Network("RL")
val queue = Queue(model, "Queue", SchedStrategy.FCFS)

In [3]:
// Block 2: Create three closed classes
val jobClass1 = ClosedClass(model, "Class1", 1, queue)  // 1 job starts in Class1
val jobClass2 = ClosedClass(model, "Class2", 0, queue)  // 0 jobs start in Class2
val jobClass3 = ClosedClass(model, "Class3", 0, queue)  // 0 jobs start in Class3

In [4]:
// Set different Erlang service distributions for each class
queue.setService(jobClass1, Erlang.fitMeanAndOrder(1.0, 2))  // Mean=1, Order=2
queue.setService(jobClass2, Erlang.fitMeanAndOrder(2.0, 2))  // Mean=2, Order=2  
queue.setService(jobClass3, Erlang.fitMeanAndOrder(3.0, 2))  // Mean=3, Order=2

In [5]:
// Block 3: Define class switching topology (Class1→Class2→Class3→Class1)
val P = model.initRoutingMatrix()
P.set(jobClass1, jobClass2, queue, queue, 1.0)  // Class1 → Class2
P.set(jobClass2, jobClass3, queue, queue, 1.0)  // Class2 → Class3
P.set(jobClass3, jobClass1, queue, queue, 1.0)  // Class3 → Class1
model.link(P)

In [6]:
// Block 4: Solve using NC solver and display results
println("=== Station-level metrics ===")
NC(model).avgTable.print()

=== Station-level metrics ===
Station  JobClass  QLen        Util        RespT       ResidT      ArvR        Tput        
-------------------------------------------------------------------------------------------
Queue    Class1    0.16667     0.16667     1.00000     0.33333     0.16667     0.16667     
Queue    Class2    0.33333     0.33333     2.00000     0.66667     0.16667     0.16667     
Queue    Class3    0.50000     0.50000     3.00000     1.00000     0.16667     0.16667     
-------------------------------------------------------------------------------------------


In [7]:
// Display system-level metrics
println("\n=== System-level metrics ===")
NC(model).avgSysTable.print()


=== System-level metrics ===
Chain   JobClasses                      SysRespT    SysTput     
------------------------------------------------------------
Chain1  (Class1 Class2 Class3)          2.00000     0.50000     
------------------------------------------------------------


In [8]:
// Block 5: Demonstrate effect of setCompletes(false)
jobClass1.setCompletes(false)  // Don't count completions for Class1
jobClass2.setCompletes(false)  // Don't count completions for Class2

println("\n=== System-level metrics (with setCompletes(false)) ===")
NC(model).avgSysTable.print()


=== System-level metrics (with setCompletes(false)) ===
Chain   JobClasses                      SysRespT    SysTput     
------------------------------------------------------------
Chain1  (Class1 Class2 Class3)          6.00000     0.16667     
------------------------------------------------------------
